In [1]:
import numpy as np
import pandas as pd
import math

import seaborn as sns
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from scipy.stats import chi2_contingency
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [10]:
train = pd.read_csv('./data/train.csv', index_col='id')
test = pd.read_csv('./data/test.csv', index_col='id')
submission = pd.read_csv('./data/sample_submission.csv', index_col='id')

In [11]:
train.columns

Index(['X00', 'X01', 'X02', 'X03', 'X04', 'X05', 'X06', 'X07', 'X08', 'X09',
       'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19',
       'X20', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29',
       'X30', 'X31', 'X32', 'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39',
       'Y00', 'Y01', 'Y02', 'Y03', 'Y04', 'Y05', 'Y06', 'Y07', 'Y08', 'Y09',
       'Y10', 'Y11', 'Y12', 'Y13', 'Y14', 'Y15', 'Y16', 'Y17', 'Y18'],
      dtype='object')

In [17]:
trainNotNull = train[train['Y18'].notnull()]
trainNotNull = trainNotNull.drop(['Y00', 'Y01', 'Y02', 'Y03', 'Y04', 'Y05', 'Y06', 'Y07', 'Y08', 'Y09', 'Y10', 'Y11', 'Y12', 'Y13', 'Y14', 'Y15', 'Y16', 'Y17'], axis=1)
trainNotNull.head()

,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,...,X31,X32,X33,X34,X35,X36,X37,X38,X39,Y18
id,,,,,,,,,,,,,,,,,,,,,
4320,19.3,987.7,0.9,2.2,0.0,1007.7,988.0,20.8,1007.8,1007.4,...,18.0,18.7,1008.3,0.0,353.5,0.0,84.0,77.6,0.0,20.5
4321,19.0,987.6,1.9,2.2,0.0,1007.7,988.0,20.6,1007.8,1007.4,...,17.7,19.0,1008.3,0.0,303.1,0.0,84.4,79.2,0.0,20.5
4322,19.1,987.6,2.0,1.4,0.0,1007.8,988.1,20.5,1007.9,1007.4,...,17.4,19.2,1008.3,0.0,307.3,0.0,84.9,79.9,0.0,20.5
4323,19.2,987.7,1.8,1.5,0.0,1007.9,988.1,20.5,1007.9,1007.5,...,17.5,19.3,1008.4,0.0,315.6,0.0,84.9,80.7,0.0,20.5
4324,19.2,987.8,1.4,1.4,0.0,1007.9,988.1,20.5,1007.8,1007.6,...,17.4,19.5,1008.4,0.0,321.2,0.0,84.9,80.9,0.0,20.5


In [18]:
columnList = trainNotNull.columns
target_var = ['Y18']
input_var = list(filter(lambda x:x != 'Y18', columnList))

In [19]:
X_train = trainNotNull[input_var]
y_train = trainNotNull[target_var]

In [20]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

param = {
    'max_depth':[2,3,4],
    'n_estimators':range(550,700,50),
    'colsample_bytree':[0.5,0.7,1],
    'colsample_bylevel':[0.5,0.7,1],
}
model = xgb.XGBRegressor()
grid_search = GridSearchCV(estimator=model, param_grid=param, cv=5, 
                           scoring='neg_mean_squared_error',
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
print(grid_search.best_estimator_)

[13:58:21] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
{'colsample_bylevel': 1, 'colsample_bytree': 1, 'max_depth': 3, 'n_estimators': 650}
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=650,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)


In [26]:
xgbr = xgb.XGBRegressor(n_estimators=1000, learning_rate=0.05)
xgbr.fit(X_train, y_train, eval_set=[(X_train, y_train)], verbose=True, early_stopping_rounds=5)

rmse:0.176885
[392]	validation_0-rmse:0.176085
[393]	validation_0-rmse:0.175379
[394]	validation_0-rmse:0.175273
[395]	validation_0-rmse:0.17445
[396]	validation_0-rmse:0.17396
[397]	validation_0-rmse:0.173617
[398]	validation_0-rmse:0.173323
[399]	validation_0-rmse:0.172784
[400]	validation_0-rmse:0.17258
[401]	validation_0-rmse:0.17238
[402]	validation_0-rmse:0.171726
[403]	validation_0-rmse:0.171007
[404]	validation_0-rmse:0.170281
[405]	validation_0-rmse:0.169871
[406]	validation_0-rmse:0.16964
[407]	validation_0-rmse:0.169397
[408]	validation_0-rmse:0.169007
[409]	validation_0-rmse:0.168396
[410]	validation_0-rmse:0.16783
[411]	validation_0-rmse:0.167715
[412]	validation_0-rmse:0.167245
[413]	validation_0-rmse:0.167066
[414]	validation_0-rmse:0.166057
[415]	validation_0-rmse:0.16542
[416]	validation_0-rmse:0.16504
[417]	validation_0-rmse:0.164929
[418]	validation_0-rmse:0.16473
[419]	validation_0-rmse:0.164007
[420]	validation_0-rmse:0.163869
[421]	validation_0-rmse:0.163659
[422]

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [28]:
testPred = xgbr.predict(test)

In [29]:
testPred

array([20.748215, 20.720142, 20.782335, ..., 27.94317 , 27.43327 ,
       27.281336], dtype=float32)

In [30]:
outputDf = pd.DataFrame()
outputDf['id'] = test.index
outputDf['Y18'] = testPred

In [31]:
outputDf.to_csv('submission.csv', index=False)